In [1]:
import torch
import torchvision

In [2]:
print("torch:", torch.__version__)

torch: 2.3.1+cu121


In [3]:
# loading the datasets
from torchvision.datasets import MNIST

In [4]:
data_train = MNIST(root="./", download=True, train=True)
data_test = MNIST(root="./", download=True, train=False)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 9912422/9912422 [00:12<00:00, 813192.70it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 56270.11it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1648877/1648877 [00:01<00:00, 1092305.43it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 4466712.49it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [5]:
data_train[10][0]

In [6]:
data_train[10][1]

3

In [7]:
type(data_train[10][0])

PIL.Image.Image

In [8]:
print(data_train[0][0].size)

(28, 28)


In [9]:
from torchvision.transforms import Lambda, ToTensor, Compose
transform = Compose([
    ToTensor(),
    Lambda(lambda image: image/255),
    Lambda(lambda image: image.view(784))
])

data_train = MNIST(root="./", download=True, train=True, transform = transform)
data_test = MNIST(root="./", download=True, train=False, transform = transform)


In [10]:
data_train[0][0].shape

torch.Size([784])

In [11]:
# Build the model
from torch import nn, optim

class Model(nn.Module):
  def __init__(self, sizes):   # [128,64,10]
    super().__init__()
    self.hidden_layer_1 = nn.Linear(784, sizes[0])  # 784,128
    self.act1 = nn.Sigmoid()
    self.hidden_layer_2 = nn.Linear(sizes[0], sizes[1])   # 128,64
    self.act2 = nn.Sigmoid()
    self.output_layer = nn.Linear(sizes[1], sizes[2])   # 64, 10
    self.act3 = nn.Softmax(dim=1)

    self.loss = nn.CrossEntropyLoss()
    self.optimizer = optim.Adam(self.parameters())

  def forward(self, inputs):
    x = self.hidden_layer_1(inputs)
    x = self.act1(x)
    x = self.hidden_layer_2(x)
    x = self.act2(x)
    x = self.output_layer(x)
    x = self.act3(x)
    return x

  def fit(self, X,Y):
    self.optimizer.zero_grad()
    y_pred = self.forward(X)
    loss = self.loss(y_pred, Y)
    loss.backward()   # calculation of gradients - backpropagation
    self.optimizer.step()   # updating the weights
    return loss.item()

  def predict(self,X):
    with torch.no_grad():
      return torch.argmax(self.forward(X), axis=1)

In [25]:
 def evaluate(self,test_dataloader):
    correct = 0
    for X,Y in test_dataloader:
      y_pred = self.predict(X)
      correct += (Y == y_pred).sum()   # batch
    acc = correct / (len(test_dataloader)* 16)
    print(f"Accuracy: {acc}")

In [12]:
model = Model([128,64,10])

In [13]:
from torch.utils.data import DataLoader
BATCH = 16
train_dataloader = DataLoader(data_train, batch_size=BATCH, shuffle=True)
test_dataloader = DataLoader(data_test, batch_size=BATCH, shuffle=True)

In [14]:
from tqdm import tqdm
epochs = 10
for i in range(epochs):
  total_loss = 0
  for X,Y in tqdm(train_dataloader, desc = f"Fitting epoch {i}"):
    loss = model.fit(X,Y)
    total_loss += loss
  avg_loss = total_loss/len(train_dataloader)
  print(f"Epoch {i}: {avg_loss:.4f}")

Fitting epoch 0:   0%|                                                                                                          | 0/3750 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/autograd/graph.py:744: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11080). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Fitting epoch 0: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 3750/3750 [00:08<00:00, 422.61it/s]


Epoch 0: 2.1708


Fitting epoch 1: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 3750/3750 [00:08<00:00, 420.77it/s]


Epoch 1: 1.9478


Fitting epoch 2: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 3750/3750 [00:08<00:00, 423.49it/s]


...

Epoch 7: 1.6940


Fitting epoch 8: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 3750/3750 [00:09<00:00, 414.25it/s]


Epoch 8: 1.6568


Fitting epoch 9: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 3750/3750 [00:08<00:00, 419.23it/s]

Epoch 9: 1.6371


In [15]:
model.evaluate(test_dataloader)

Accuracy: 0.8439000248908997


In [16]:
path = "./mnist_model.pth"
torch.save(model.state_dict(), path)  # only weights


In [17]:
model_2 = Model([128,64,10])
model_2.load_state_dict(torch.load(path))

<All keys matched successfully>

In [18]:
model_2.evaluate(test_dataloader)

Accuracy: 0.8439000248908997


In [19]:
path = "./full_model.pth"
torch.save(model, path)  # full model with arch and weights


In [20]:
model_3 = torch.load(path)

In [21]:
model_3.evaluate(test_dataloader)

Accuracy: 0.8439000248908997


In [22]:
for X,Y in test_dataloader:
  y_pred = model.predict(X)
  print(Y, y_pred)
  print(Y==y_pred)
  break

tensor([3, 7, 0, 7, 2, 5, 4, 3, 4, 7, 3, 7, 1, 1, 3, 2]) tensor([3, 3, 5, 8, 6, 5, 4, 3, 9, 7, 3, 7, 1, 1, 3, 2])
tensor([ True, False, False, False, False,  True,  True,  True, False,  True,
         True,  True,  True,  True,  True,  True])
